In [1]:
import cv2
import numpy as np
import sys

In [10]:
# 트랙바 콜백 함수 생성
def nothing(x):
    pass

def on_trackbar():
    global hmin,hmax,smin,smax,vmin,vmax
    hmin = cv2.getTrackbarPos('H_min', 'dst')
    hmax = cv2.getTrackbarPos('H_max', 'dst')
    smin = cv2.getTrackbarPos('S_min', 'dst')
    smax = cv2.getTrackbarPos('S_max', 'dst')
    vmin = cv2.getTrackbarPos('V_min', 'dst')
    vmax = cv2.getTrackbarPos('V_max', 'dst')

In [11]:
# cv2.namedWindow('src')
cv2.namedWindow('dst')

# 트랙바 생성
cv2.createTrackbar('H_min', 'dst', 0, 255, nothing)
cv2.setTrackbarPos('H_min', 'dst',60)
cv2.createTrackbar('H_max', 'dst', 0, 255, nothing)
cv2.setTrackbarPos('H_max', 'dst',100)
cv2.createTrackbar('S_min', 'dst', 0, 255, nothing)
cv2.setTrackbarPos('S_min', 'dst',60)
cv2.createTrackbar('S_max', 'dst', 0, 255, nothing)
cv2.setTrackbarPos('S_max', 'dst',100)
cv2.createTrackbar('V_min', 'dst', 0, 255, nothing)
cv2.setTrackbarPos('V_min', 'dst',60)
cv2.createTrackbar('V_max', 'dst', 0, 255, nothing)
cv2.setTrackbarPos('V_max', 'dst',100)

In [12]:
while(True):
    #set the cam num
    cam_num=0
    src = cv2.imread(f'./cam{cam_num}_ex.jpg')

    #이미지 불러와졌는지
    if src is None:
        print('Image load failed!')
        sys.exit()
    
    cv2.imshow('src', src)
    src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
    
    #Trackbar 불러오기
    on_trackbar()
    
    #Detecting Color Setting
    dst = cv2.inRange(src_hsv, (hmin, smin, vmin), (hmax, smax, vmax))
    cv2.imshow('dst', dst)
    
    #MORPH 함수 이용하여 정확도 향상
    kernel = np.ones((11,11), np.uint8)
    dst = cv2.morphologyEx(dst, cv2.MORPH_OPEN, kernel)
    dst = cv2.morphologyEx(dst, cv2.MORPH_CLOSE, kernel)

    # 마스크 이미지로 원본 이미지에서 범위값에 해당되는 영상 부분을 획득
    img_result = cv2.bitwise_and(src, src, mask=dst)

    cv2.imshow('src', src)
    cv2.imshow('dst',dst)
    cv2.imshow('img_result', img_result)
    
    if cv2.waitKey(1) & 0xFF == 27:
        cv2.imwrite(f'cam{cam_num}_mask_cali_v2.jpg', ~dst)
        break
    
cv2.destroyAllWindows()

In [5]:
#color inversion
src = cv2.imread('cam0_mask_cali_v2.jpg')

out=src.copy()
out=255-out

src=out


In [3]:
import cv2 as cv
 
# ROI 마스크 PNG 파일 추출

print("in draw_mask")
pts = []  # 마우스로 클릭한 포인트 저장
mask_list = []  # 마스크 리스트 저장

# 마스크 추출을 위한 마우스 클릭 이벤트 리스너
def draw_mask_eventListener(event, x, y, flags, param):
    global pts
    img2 = img.copy()

    if event == cv.EVENT_LBUTTONDOWN:  # 마우스 왼쪽 버튼 클릭 시 pts에 (x,y)좌표를 추가
        pts.append((x, y))

    if event == cv.EVENT_RBUTTONDOWN:  # 마우스 오른쪽 버튼 클릭 시 클릭 했던 포인트를 삭제
        pts.pop()

    if event == cv.EVENT_LBUTTONDBLCLK:  # 마우스 왼쪽 버튼 더블 클릭 시 좌표들을 리스트에 추가
        # 초기화
        mask_list.append(pts)
        pts = []

    if event == cv.EVENT_MBUTTONDOWN:  # 마우스 중앙(휠)버튼 클릭 시 ROI 선택 종료
        result_roi = np.zeros(img.shape, np.uint8)  # 최종 마스크 이미지

        for point in mask_list:
            if not point: continue
            mask = np.zeros(img.shape, np.uint8)
            points = np.array(point, np.int32)
            points = points.reshape((-1, 1, 2))  # pts 2차원을 이미지와 동일하게 3차원으로 재배열
            mask = cv.polylines(mask, [points], True, (255, 255, 255), 2)  # 포인트를 연결하는 라인을 설정 후 마스크 생성
            mask2 = cv.fillPoly(mask.copy(), [points], (255, 255, 255))  # 채워진 다각형 마스크 생성

            ROI = cv.bitwise_and(mask2, img)  # img와 mask2에 중첩된 부분을 추출

            result_roi = cv.add(result_roi, ROI)  # 마스크 이미지끼리 더하기

        result_roi = np.where(result_roi == 0, result_roi, 255)  # 첫번째 매개변수 조건에 따라 참이면 유지, 거짓이면 255으로 변경
        cv.imwrite('cam1_mask_cali_v2.jpg', result_roi) # 저장
        cv.destroyAllWindows()  # 열린 창 닫기
        cv.waitKey(0)

    try:
        if len(pts) > 0:  # 마우스 포인트 원으로 지정
            cv.circle(img2, pts[-1], 3, (0, 0, 255), -1)
    except:
        pts = []

    if len(pts) > 1:  # 마우스 포인트 연결 라인 생성
        for i in range(len(pts) - 1):
            cv.circle(img2, pts[i], 5, (0, 0, 255), -1)
            cv.line(img=img2, pt1=pts[i], pt2=pts[i + 1], color=(255, 0, 0), thickness=2)

    if len(mask_list) > 0:  # 마스크 여러 개일때 포인트 연결 라인 생성
        for m in mask_list:
            for i in range(len(m) - 1):
                cv.circle(img2, m[i], 5, (0, 0, 255), -1)
                cv.line(img=img2, pt1=m[i], pt2=m[i + 1], color=(255, 0, 0), thickness=2)

    cv.imshow('image', img2)  # 이미지 화면 출력

img = cv.imread('./cam1_ex.jpg')
cv.startWindowThread()
cv.namedWindow('image')  # 새로운 윈도우 창 이름 설정
cv.setMouseCallback('image', draw_mask_eventListener)  # 마우스 이벤트가 발생했을 때 전달할 함수

while True:
    key = cv.waitKey(1) & 0xFF  # SOH
    if key == 27:  # ESC
        break
cv.destroyAllWindows()  # 열린 창 닫기

in draw_mask
